In [2]:
import numpy as np

G = np.array([
    [1, 1, 0, 1, 0],
    [0, 1, 1, 0, 1],
    [1, 0, 1, 1, 0]
], dtype=int)


def w(x: np.ndarray):
    return np.count_nonzero(x)

k, n = G.shape
r = n - k

def generate_code_words_with_info_bits(G):
    code_words = []
    info_bits_list = []
    for i in range(2**k):
        info_bits = np.array([int(x) for x in f"{i:0{k}b}"], dtype=int)
        code_word = np.mod(np.dot(info_bits, G), 2)
        code_words.append(code_word)
        info_bits_list.append(info_bits)
    return np.array(code_words), np.array(info_bits_list)

code_words, info_bits = generate_code_words_with_info_bits(G)

print("Кодовые слова и соответствующие им исходные биты:")
dmin = n + 1
for i in range(len(code_words)):
    print(f"{info_bits[i]} : {code_words[i]}")
    wi = w(code_words[i])
    if wi < dmin and wi != 0:
        dmin = wi

print(f'n = {n}, k = {k}, d_min = {dmin}, t = {np.floor((dmin - 1) / 2)}')

GforH = np.array(
    code_words[[5, 4, 7]]
)

print(f'Custom:\n {GforH}')

H = np.hstack((np.eye(r, dtype=int), GforH[:, :2].T))
print('Проверочная матрица:')
print(H)

print('Проверка:')
print(np.mod(G @ H.T, 2))

def build_syndrome_table(H, n):
    syndrome_table = {}
    for i in range(2**n):
        error_vector = np.array([int(x) for x in f"{i:0{n}b}"], dtype=int)
        # print(f'error vector: {error_vector}')
        syndrome = np.mod(np.dot(H, error_vector) % 2, 2)
        syndrome_tuple = tuple(syndrome)
        if syndrome_tuple not in syndrome_table:
            syndrome_table[syndrome_tuple] = error_vector
        elif w(error_vector) < w(syndrome_table[syndrome_tuple]):
            syndrome_table[syndrome_tuple] = error_vector
    return syndrome_table

syndrome_table = build_syndrome_table(H, n)

print("Таблица синдромного декодирования:")
for syndrome, error in syndrome_table.items():
    print(f"Синдром {syndrome}: Ошибка {error}")

Кодовые слова и соответствующие им исходные биты:
[0 0 0] : [0 0 0 0 0]
[0 0 1] : [1 0 1 1 0]
[0 1 0] : [0 1 1 0 1]
[0 1 1] : [1 1 0 1 1]
[1 0 0] : [1 1 0 1 0]
[1 0 1] : [0 1 1 0 0]
[1 1 0] : [1 0 1 1 1]
[1 1 1] : [0 0 0 0 1]
n = 5, k = 3, d_min = 1, t = 0.0
Custom:
 [[0 1 1 0 0]
 [1 1 0 1 0]
 [0 0 0 0 1]]
Проверочная матрица:
[[1 0 0 1 0]
 [0 1 1 1 0]]
Проверка:
[[0 0]
 [0 0]
 [0 0]]
Таблица синдромного декодирования:
Синдром (0, 0): Ошибка [0 0 0 0 0]
Синдром (1, 1): Ошибка [0 0 0 1 0]
Синдром (0, 1): Ошибка [0 0 1 0 0]
Синдром (1, 0): Ошибка [1 0 0 0 0]


In [3]:
print('Пример:')
y = np.random.randint(2, size=n)
print(f'Полученное сообщение: {y}')
s = np.mod(np.dot(H, y), 2)
print(f'Синдром: {s}')
e = syndrome_table[tuple(s)]
print(f'Позиция в таблице: {e}')
print(f'Предполагаемое сообщение: {np.mod(y + e, 2)}')

Пример:
Полученное сообщение: [0 1 0 1 0]
Синдром: [1 0]
Позиция в таблице: [1 0 0 0 0]
Предполагаемое сообщение: [1 1 0 1 0]
